In [ ]:
import os, sys, tqdm
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"#"last_expr" #all for debugging better
pd.set_option('display.max_columns', 500,'display.max_rows',200,'display.max_colwidth',200)

In [ ]:
import dropbox
import cv2

In [ ]:
file = '' # path to csv file, MUST HAVE "path" column with pathes to images to be uploaded
target = '' # target path where to save new csv with URLs
df = pd.read_csv(file, index_col=0)

access_token = '' # get access token from dropbox
folder = '' # dropbox subfolder, can be empty string

In [ ]:
dbx = dropbox.Dropbox(access_token)

In [ ]:
[i.name for i in dbx.files_list_folder(folder).entries] # will print list with all files which are already in your dropbox folder

In [ ]:
# These errors will occur
for i in df.index:
    try:
        path = df.loc[i,'path']
        _=open(path, "rb").read()
    except:
        print(sys.exc_info()[0],df.loc[i,'path'])

In [ ]:
for i in tqdm.tqdm(df.index):
    try:
        path = df.loc[i,'path']
        _ = dbx.files_upload(open(path, "rb").read(), '/'+path.split('/')[-1])
        url = dbx.sharing_create_shared_link(path='/'+path.split('/')[-1]).url
        df.loc[i,'image_url'] = url
    #     dbx.sharing_list_shared_links('/'+i.split('/')[-1]).links[0].url
    except:
        print(df.loc[i,'path'],sys.exc_info()[0])
        
df['image_url']=df.image_url.str.replace('dl=0','raw=1')
df.iloc[:,[0]].dropna().to_csv(target, index=False)